[View in Colaboratory](https://colab.research.google.com/github/sabumjung/DL-with-TensorFlow/blob/DL-with-TensorFlow/8%EC%9E%A5_2.ipynb)

In [0]:
# 필요한 패키지를 불러온다.
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D

In [0]:
# 재현을 위한 랜덤시드를 설정한다.
numpy.random.seed(7)

In [4]:
# 데이터 집합을 불러와 톱 n개의 단어만 보관하고, 나머지는 0으로 설정한다.
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=top_words)

# 입력 시퀀스를 최대길이 500으로 맞춘다.(부족한 경우 0으로 설정하여 패딩처리한다.)
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

/usr/local/lib/python3.6/dist-packages/keras/datasets/imdb.py:49: UserWarning: The `nb_words` argument in `load_data` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `load_data` '


17465344/17464789 [==============================] - 11s 1us/step


In [5]:
print(X_train, X_test)
print(y_train, y_test)
print(len(X_train), len(X_test))

[[   0    0    0 ...   19  178   32]
 [   0    0    0 ...   16  145   95]
 [   0    0    0 ...    7  129  113]
 ...
 [   0    0    0 ...    4 3586    2]
 [   0    0    0 ...   12    9   23]
 [   0    0    0 ...  204  131    9]] [[   0    0    0 ...   14    6  717]
 [   0    0    0 ...  125    4 3077]
 [  33    6   58 ...    9   57  975]
 ...
 [   0    0    0 ...   21  846    2]
 [   0    0    0 ... 2302    7  470]
 [   0    0    0 ...   34 2005 2643]]
[1 0 0 ... 0 1 0] [0 1 1 ... 0 0 0]
25000 25000


In [10]:
# 임베딩 벡터길이를 32로 설정한다.
embedding_vector_length=32

# 해당 신경망 모델을 생성한다.
model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
model.add(Conv1D(padding="same", activation="relu", kernel_size=3, filters=32))
model.add(MaxPooling1D(pool_size=2))

model.add(LSTM(32, input_dim=64, return_sequences=True))
model.add(LSTM(24, return_sequences=True))
model.add(LSTM(1,  return_sequences=False))

model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(32, return_sequences=True, input_shape=(None, 64))`
  if __name__ == '__main__':


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 500, 32)           3104      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 250, 32)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 250, 32)           8320      
_________________________________________________________________
lstm_6 (LSTM)                (None, 250, 24)           5472      
_________________________________________________________________
lstm_7 (LSTM)                (None, 1)                 104       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 2         
Total para

In [11]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=3, batch_size=6400)

/usr/local/lib/python3.6/dist-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 22s 870us/step - loss: 0.6932 - acc: 0.4994 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 2/3
25000/25000 [==============================] - 18s 711us/step - loss: 0.6931 - acc: 0.5000 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 3/3
25000/25000 [==============================] - 18s 712us/step - loss: 0.6931 - acc: 0.5000 - val_loss: 0.6931 - val_acc: 0.5000


In [12]:
# 모델에 대한 최종 평가를 한다.
scores = model.evaluate(X_test, y_test, verbose=1)

print("Accuracy: %.2f%%" % (scores[1]*100))

25000/25000 [==============================] - 324s 13ms/step
Accuracy: 50.00%
